In [1]:
 // @hidden_cell
 // In a Project create a Connection to a Message Hub topic and then in a Notebook in the Project use 'Insert to Code' to get the connection info pasted into the code cell.
 // Just like this: 
var credentials_1 = scala.collection.mutable.HashMap[String, String](
  "instance_id"->"***",
  "mqlight_lookup_url"->"***",
  "api_key"->"***",
  "kafka_admin_url"->"***",
  "kafka_rest_url"->"***",
  "kafka_brokers_sasl"->"ip:port",
  "user"->"***",
  "password"->"""****""",
  "topic"->"test"
)
spark.version

Waiting for a Spark session to start...

2.1.0

In [2]:
import scala.util.Try


In [3]:
case class Click(prev: String, curr: String, link: String, n: Long)


In [4]:
def parseVal(x: Array[Byte]): Option[Click] = {
    val split: Array[String] = new Predef.String(x).split("\\t")
    if (split.length == 4) {
      Try(Click(split(0), split(1), split(2), split(3).toLong)).toOption
    } else
      None
  }

In [7]:
val user = credentials_1("user")


In [8]:
println(user)

***


In [9]:
val pass = credentials_1("password")


In [10]:
println(pass)

***


In [11]:
 val saslConfig =
    s"""org.apache.kafka.common.security.plain.PlainLoginModule required
      |debug=true
      |username="$user"
      |password="$pass";""".stripMargin


In [12]:
println(saslConfig)

org.apache.kafka.common.security.plain.PlainLoginModule required
debug=true
username="***"
password="***";


In [13]:
val records = spark.readStream.format("kafka").option("kafka.sasl.jaas.config", saslConfig).option("subscribe", "test").option("kafka.sasl.mechanism", "PLAIN").option("kafka.security.protocol", "SASL_SSL").option("failOnDataLoss", "false").option("kafka.bootstrap.servers", credentials_1("kafka_brokers_sasl")).load()


In [14]:
val messages = records.select("value").as[Array[Byte]].flatMap(x => parseVal(x)).groupBy("curr").agg(Map("n" -> "sum")).sort($"sum(n)".desc)

In [15]:
val query = messages.writeStream.outputMode("complete").option("truncate", "false").format("console").start()


-------------------------------------------
Batch: 0
-------------------------------------------
+----+------+
|curr|sum(n)|
+----+------+
+----+------+

-------------------------------------------
Batch: 7
-------------------------------------------
+---------+------+
|curr     |sum(n)|
+---------+------+
|Aristotle|108441|
+---------+------+

-------------------------------------------
Batch: 8
-------------------------------------------
+--------------------------------------------+------+
|curr                                        |sum(n)|
+--------------------------------------------+------+
|Aristotle                                   |108441|
|Monday_Night_Massacre                       |49366 |
|Fin_(Syd_album)                             |762   |
|#################                           |313   |
|Binghamton_Devils                           |280   |
|Fueler                                      |254   |
|Alpine_skiing_at_the_2017_Winter_Universiade|193   |
|Mudyug_concentr